In [1]:
import pandas as pd
import numpy as np

# plot
import plotly.express as px
import plotly.graph_objects as go

#Import machine learning
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
# import xgboost

from sklearn.model_selection import train_test_split #split
from sklearn.metrics import r2_score, mean_squared_error #metrics

# Data Extraction

In [48]:
listings = pd.read_csv('../data/airbnb_sydney/listings.csv') # list of info about the house, location, host, etc.
# reviews = pd.read_csv('../data/airbnb_sydney/reviews.csv') # reviews => details of reviews in listings_df
# calendar = pd.read_csv('../data/airbnb_sydney/calendar.csv') # calendar (house availbility)

# Data Understainding

In [4]:
# 1. NULL checking
listings.isnull().mean()

id                                              0.000000
listing_url                                     0.000000
scrape_id                                       0.000000
last_scraped                                    0.000000
name                                            0.000367
                                                  ...   
calculated_host_listings_count                  0.000000
calculated_host_listings_count_entire_homes     0.000000
calculated_host_listings_count_private_rooms    0.000000
calculated_host_listings_count_shared_rooms     0.000000
reviews_per_month                               0.294042
Length: 74, dtype: float64

In [10]:
listings.columns

Index(['id', 'listing_url', 'scrape_id', 'last_scraped', 'name', 'description',
       'neighborhood_overview', 'picture_url', 'host_id', 'host_url',
       'host_name', 'host_since', 'host_location', 'host_about',
       'host_response_time', 'host_response_rate', 'host_acceptance_rate',
       'host_is_superhost', 'host_thumbnail_url', 'host_picture_url',
       'host_neighbourhood', 'host_listings_count',
       'host_total_listings_count', 'host_verifications',
       'host_has_profile_pic', 'host_identity_verified', 'neighbourhood',
       'neighbourhood_cleansed', 'neighbourhood_group_cleansed', 'latitude',
       'longitude', 'property_type', 'room_type', 'accommodates', 'bathrooms',
       'bathrooms_text', 'bedrooms', 'beds', 'amenities', 'price',
       'minimum_nights', 'maximum_nights', 'minimum_minimum_nights',
       'maximum_minimum_nights', 'minimum_maximum_nights',
       'maximum_maximum_nights', 'minimum_nights_avg_ntm',
       'maximum_nights_avg_ntm', 'calendar_upd

In [18]:
listings[listings['id'] == 49060109][['neighbourhood_cleansed', 'availability_30', 'availability_60']]

,neighbourhood_cleansed,availability_30,availability_60
32642,Warringah,10,13


In [17]:
calendar_df[(calendar['listing_id'] == 49060109) & 
            (calendar['available'] == 't') & 
            (calendar['date'] < '2021-05-12')]

,listing_id,date,available,price,adjusted_price,minimum_nights,maximum_nights
11930738,49060109,2021-04-17,t,$420.00,$420.00,1.0,1125.0
11930739,49060109,2021-04-18,t,$350.00,$350.00,1.0,1125.0
11930740,49060109,2021-04-19,t,$350.00,$350.00,1.0,1125.0
11930741,49060109,2021-04-20,t,$350.00,$350.00,1.0,1125.0
11930742,49060109,2021-04-21,t,$350.00,$350.00,1.0,1125.0
11930743,49060109,2021-04-22,t,$350.00,$350.00,1.0,1125.0
11930744,49060109,2021-04-23,t,$420.00,$420.00,1.0,1125.0
11930745,49060109,2021-04-24,t,$420.00,$420.00,1.0,1125.0
11930746,49060109,2021-04-25,t,$350.00,$350.00,1.0,1125.0
11930747,49060109,2021-04-26,t,$350.00,$350.00,1.0,1125.0


# Data Cleaning 

In [49]:
listings_df = listings[[
    'id', 'name', 'neighbourhood_cleansed','latitude', 'longitude', 
    'property_type', 'room_type', 'accommodates', 
    'bathrooms_text', 'bedrooms', 'beds', 'amenities', 'price',
    'minimum_nights', 'maximum_nights', 'has_availability',
    'availability_30', 'availability_60', 'availability_90',
    'availability_365', 'instant_bookable',
    'number_of_reviews',
    'number_of_reviews_ltm', 'number_of_reviews_l30d',
    'review_scores_rating', 'review_scores_accuracy',
    'review_scores_cleanliness', 'review_scores_checkin',
    'review_scores_communication', 'review_scores_location',
    'review_scores_value', 'reviews_per_month', 
    'host_id',
    'host_name', 'host_since', 'host_location',
    'host_acceptance_rate', 'host_is_superhost',
    'host_neighbourhood', 'host_listings_count','host_verifications',
    'calculated_host_listings_count',
    'calculated_host_listings_count_entire_homes',
    'calculated_host_listings_count_private_rooms',
    'calculated_host_listings_count_shared_rooms'
]]

In [56]:
listings_df.isnull().mean()

id                                              0.000000
name                                            0.000367
neighbourhood_cleansed                          0.000000
latitude                                        0.000000
longitude                                       0.000000
property_type                                   0.000000
room_type                                       0.000000
accommodates                                    0.000000
bathrooms_text                                  0.000918
bedrooms                                        0.065730
beds                                            0.012577
amenities                                       0.000000
price                                           0.000000
minimum_nights                                  0.000000
maximum_nights                                  0.000000
has_availability                                0.000000
availability_30                                 0.000000
availability_60                

In [54]:
listings_df.loc[:, 'price'] = listings_df['price'].map(lambda p: int(p[1:-3].replace(",", "")))
listings_df.loc[:, 'amenities'] = listings_df['amenities'].map(
    lambda amns: "|".join([amn.replace("]", "").replace("[", "").replace('"', "").strip()\
                           for amn in amns.split(",")])
)

<ipython-input-54-ea36b6bdac93>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  listings_df['price'] = listings_df['price'].map(lambda p: int(p[1:-3].replace(",", "")))
<ipython-input-54-ea36b6bdac93>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  listings_df['amenities'] = listings_df['amenities'].map(


In [110]:
listings_df[(listings_df['has_availability'] == 'f') & (listings_df['availability_30'] > 0)][[
    'minimum_nights', 'maximum_nights',
    'availability_30', 'availability_60', 'availability_90',
    'availability_365',
]]
# 证明了has_availability = f的房子 已经不再出租了 可以认为是outlier

,minimum_nights,maximum_nights,availability_30,availability_60,availability_90,availability_365


In [103]:
listings_df['room_type'].unique()

array(['Private room', 'Entire home/apt', 'Shared room', 'Hotel room',
       nan], dtype=object)

In [129]:
listings_df = listings_df[listings_df['has_availability'] == 't']

### Pose at least three questions related to business or real-world applications of how the data could be used.
make it easy !!!
1. which neighbourhood gets the most revenue? which type of room type get most revenue?
2. which service in house is customers most concerned - reviews & marks
3. find the hottest place in Sydeny in the next month - the LGA most most booked and the rate

4. BK: how to predict the price

##### 1. which neighbourhood gets the most revenue? which type of room type get most revenue?

In [146]:
df1 = listings_df[[
    'id', 'name', 'neighbourhood_cleansed','latitude', 'longitude', 
    'property_type', 'room_type', 'accommodates', 
    'bathrooms_text', 'bedrooms', 'beds', 'amenities', 'price', 'minimum_nights',
    'availability_30'
]]

df1.loc[:, 'price_per_night'] = round(df1['price'] / df1['minimum_nights'], 2)
df1.loc[:, 'revenues_30'] = (30 - df1['availability_30']) * df1['price_per_night']
df1.loc[:, 'revenues_30_ideal'] = 30 * df1['price_per_night']

/Users/kevin/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:845: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/Users/kevin/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [182]:
def average_cal(values):
    return round(np.sum(values), 2)

df1_revenues30_by_room_type = pd.pivot_table(
    data=df1,
    index=['neighbourhood_cleansed'],
    columns=['room_type'],
    values=['revenues_30'],
    aggfunc=average_cal
)['revenues_30'].reset_index()

df1_tran = df1.groupby(
    ['neighbourhood_cleansed', 'room_type']
).agg(
    listing_count = ('id', 'count'),
)

df1_listing_count_by_room_type = pd.pivot_table(
    data=df1_tran,
    index=['neighbourhood_cleansed'],
    columns=['room_type'],
    values=['listing_count']
)['listing_count'].reset_index()

df1_cleansed = df1_revenues30_by_room_type.merge(
    df1_listing_count_by_room_type, 
    left_on='neighbourhood_cleansed', 
    right_on='neighbourhood_cleansed'
)


In [183]:
df1_cleansed.loc[:, 'Entire home/apt'] = round(df1_cleansed['Entire home/apt_x'] / df1_cleansed['Entire home/apt_y'], 2)
df1_cleansed.loc[:, 'Hotel room'] = round(df1_cleansed['Hotel room_x'] / df1_cleansed['Hotel room_y'], 2)
df1_cleansed.loc[:, 'Private room'] = round(df1_cleansed['Private room_x'] / df1_cleansed['Private room_y'], 2)
df1_cleansed.loc[:, 'Shared room'] = round(df1_cleansed['Shared room_x'] / df1_cleansed['Shared room_y'], 2)

df1_cleansed = df1_cleansed[[
    'neighbourhood_cleansed', 'Entire home/apt', 'Hotel room', 'Private room', 'Shared room'
]]

In [184]:
df1_cleansed

room_type,neighbourhood_cleansed,Entire home/apt,Hotel room,Private room,Shared room
0,Ashfield,1856.64,NaN,816.33,1059.28
1,Auburn,1950.11,5355.60,3690.05,1033.76
2,Bankstown,1280.91,NaN,584.05,60.60
3,Blacktown,1258.25,NaN,489.05,NaN
4,Botany Bay,2060.13,5970.00,907.25,810.38
5,Burwood,1733.35,1650.00,668.47,859.45
6,Camden,1440.42,NaN,1351.41,0.00
7,Campbelltown,1876.74,NaN,708.34,850.00
8,Canada Bay,1690.21,NaN,1677.68,969.68
9,Canterbury,1432.43,NaN,723.19,205.92


##### 2. ranking of neighbour (top 5) in review scores from total, and single service

In [96]:
df2 = listings_df[-listings_df['review_scores_value'].isnull()][[
    'id', 'neighbourhood_cleansed',
    'review_scores_cleanliness', 'review_scores_checkin',
    'review_scores_communication', 'review_scores_location',
    'review_scores_value'
]]

In [142]:
def average_cal(values):
    return round(np.mean(values), 2)

df2.groupby(
    ['neighbourhood_cleansed']
).agg(
    review_scores_avg = ('review_scores_value', average_cal),
    cleanliness_scores_avg = ('review_scores_cleanliness', average_cal),
    checkin_scores_avg = ('review_scores_checkin', average_cal),
    communication_scores_avg = ('review_scores_communication', average_cal),
    location_scores_avg = ('review_scores_location', average_cal)
).reset_index().sort_values(['review_scores_avg'], ascending=False)

,neighbourhood_cleansed,review_scores_avg,cleanliness_scores_avg,checkin_scores_avg,communication_scores_avg,location_scores_avg
6,Camden,9.61,9.52,9.84,9.87,9.77
31,Sutherland Shire,9.60,9.66,9.93,9.93,9.89
34,Warringah,9.48,9.39,9.84,9.83,9.73
23,North Sydney,9.43,9.36,9.76,9.74,9.82
25,Penrith,9.42,9.53,9.78,9.80,9.61
18,Leichhardt,9.42,9.35,9.82,9.82,9.71
21,Marrickville,9.41,9.12,9.74,9.70,9.64
13,Hornsby,9.41,9.27,9.77,9.75,9.59
26,Pittwater,9.40,9.61,9.88,9.87,9.86
19,Liverpool,9.39,9.27,9.65,9.61,9.42


##### 3. how to find the hottest place in Sydeny - the LGA most most booked and the rate

In [130]:
def stays_sum(x):
    stays = 30-x
    return stays.sum()

def booked_ratio_cal(x):
    return round(np.sum(x < 30) / np.sum(x >= 0) * 100, 2)

df = listings_df.groupby(
    ['neighbourhood_cleansed']
).agg(
    listing_count = ('id', "count"), # sum of listings
    booked_ratio = ('availability_30', booked_ratio_cal),
    stays_sum = ('availability_30', stays_sum),
    price_avg = ('price', 'mean')
).reset_index().sort_values(['booked_ratio'], ascending=False)

df.loc[:, 'stays_per_listing'] = round(df['stays_sum'] / df['listing_count'], 2)

In [135]:
df

,neighbourhood_cleansed,listing_count,booked_ratio,stays_sum,price_avg,stays_per_listing
26,Pittwater,1141,96.67,22891.0,637.921998,20.06
21,Marrickville,1095,96.44,27519.0,124.145205,25.13
35,Waverley,4605,96.40,116133.0,243.191097,25.22
37,Woollahra,1309,95.80,31858.0,361.994652,24.34
22,Mosman,458,95.63,11096.0,440.137555,24.23
20,Manly,1596,95.55,37665.0,326.416667,23.60
32,Sydney,7741,95.41,185407.0,181.645653,23.95
34,Warringah,1665,95.32,39664.0,290.437838,23.82
18,Leichhardt,771,95.20,18074.0,222.315175,23.44
23,North Sydney,1182,95.09,27185.0,211.380711,23.00


In [91]:
# def availability_ratio(x):
    

# listings_df.groupby(
#     ['neighbourhood_cleansed', 'has_availability']
# ).agg(
    
# )

df = pd.crosstab(listings_df['neighbourhood_cleansed'],
            listings_df['has_availability'],
            values=listings_df['id'],
            aggfunc='count',
            normalize=False)



has_availability,f,t
neighbourhood_cleansed,,
Ashfield,3.0,246.0
Auburn,2.0,361.0
Bankstown,4.0,247.0
Blacktown,11.0,264.0
Botany Bay,1.0,601.0
Burwood,NaN,228.0
Camden,NaN,50.0
Campbelltown,1.0,105.0
Canada Bay,2.0,373.0
